In [88]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import plotly.graph_objects as go
import plotly.express as px

In [89]:
notable_models_df = pd.read_csv("notable_ai_models.csv")
cooling_df = pd.read_csv("cooling_type_data.csv")
carbon_df = pd.read_csv("carbon_intensity_electricity.csv")
chip_df = pd.read_csv("chip_dataset.csv")

notable_models_df['Publication date'] = pd.to_datetime(notable_models_df['Publication date'], errors='coerce')
notable_models_df['Year'] = notable_models_df['Publication date'].dt.year
notable_models_df['Training compute (FLOP)'] = pd.to_numeric(notable_models_df['Training compute (FLOP)'], errors='coerce')
notable_models_df = notable_models_df.dropna(subset=['Year', 'Training compute (FLOP)'])

notable_models_df = notable_models_df[notable_models_df['Year'].between(2015, 2025)]

top_notable_models = notable_models_df.sort_values('Training compute (FLOP)', ascending=False).drop_duplicates(subset=['Year'])
top_notable_models = top_notable_models[['Model', 'Year', 'Training compute (FLOP)', 'Training hardware']].copy()
top_notable_models['Model (Year)'] = top_notable_models['Model'] + " (" + top_notable_models['Year'].astype(str) + ")"
top_notable_models['key'] = 1

cooling_df = cooling_df.drop_duplicates(subset=['Cooling System']).copy()
cooling_df = cooling_df[cooling_df['Size'] == 'Large-scale'].copy()
cooling_df['key'] = 1

combined_notable = pd.merge(top_notable_models, cooling_df, on='key').drop(columns='key')

chip_df['Release Date'] = pd.to_datetime(chip_df['Release Date'], format="%m/%d/%y", errors='coerce')
chip_df['Year'] = chip_df['Release Date'].dt.year
chip_df['FP32 GFLOPS'] = pd.to_numeric(chip_df['FP32 GFLOPS'], errors='coerce')
chip_df['TDP (W)'] = pd.to_numeric(chip_df['TDP (W)'], errors='coerce')

unique_models = top_notable_models['Model (Year)'].dropna().unique()
palette = sns.color_palette("tab10", n_colors=len(unique_models))
model_color_map = dict(zip(unique_models, palette))

In [90]:
def find_chip_info(hardware: str, year: int, df: pd.DataFrame):
    df['Release Date'] = pd.to_datetime(df['Release Date'], errors='coerce')
    df['FP32 GFLOPS'] = pd.to_numeric(df['FP32 GFLOPS'], errors='coerce')
    df['TDP (W)'] = pd.to_numeric(df['TDP (W)'], errors='coerce')

    if isinstance(hardware, str):
        matched = df[df['Product'].str.lower() == hardware.lower()]
        if not matched.empty and pd.notna(matched.iloc[0]['FP32 GFLOPS']):
            return matched.iloc[0]['FP32 GFLOPS'], matched.iloc[0]['TDP (W)']

    # Fallback to best chip in year
    chips_year = df[df['Release Date'].dt.year == year]
    chips_year = chips_year.dropna(subset=['FP32 GFLOPS'])
    if not chips_year.empty:
        best_chip = chips_year.loc[chips_year['FP32 GFLOPS'].idxmax()]
        return best_chip['FP32 GFLOPS'], best_chip['TDP (W)']

    return None, None

In [ ]:
def create_scatter_plot(data, y_column, y_label, y_units, plot_title):
    df = data.copy()
    
    unique_models = df['Model (Year)'].unique()
    unique_cooling = df['Cooling System'].unique()
    
    model_color_map = {model: f"hsl({i * 360 / len(unique_models)},70%,50%)" 
                      for i, model in enumerate(unique_models)}
    
    available_symbols = [
        'circle', 'x', 'square', 'diamond', 'cross', 'triangle-up', 'star'
    ]
    cooling_symbol_map = {cooling: available_symbols[i % len(available_symbols)] 
                         for i, cooling in enumerate(unique_cooling)}
    
    fig = go.Figure()
    
    for model in unique_models:
        model_data = df[df['Model (Year)'] == model]
        
        symbols = [cooling_symbol_map.get(cooling, 'circle') for cooling in model_data['Cooling System']]
        
        hover_text = []
        for _, row in model_data.iterrows():
            hover_text.append(f'<b>{model}</b><br>' +
                             f'Cooling: {row["Cooling System"]}<br>' +
                             f'Training Compute: {row["Training compute (FLOP)"]:.2e} FLOP<br>' +
                             f'{y_label}: {row[y_column]:.2e} {y_units}')
        
        fig.add_trace(go.Scatter(
            x=model_data['Training compute (FLOP)'],
            y=model_data[y_column],
            mode='markers',
            marker=dict(
                color=model_color_map.get(model, '#000000'),
                symbol=symbols,
                size=8,
                line=dict(width=0.5, color='white')
            ),
            name=model,
            hovertemplate='%{hovertext}<extra></extra>',
            hovertext=hover_text,
            showlegend=True
        ))
    
    for cooling in unique_cooling:
        fig.add_trace(go.Scatter(
            x=[None],
            y=[None],
            mode='markers',
            marker=dict(
                color='rgba(0,0,0,0.7)',
                symbol=cooling_symbol_map.get(cooling, 'circle'),
                size=8,
                line=dict(width=1, color='white')
            ),
            name=cooling,
            legendgroup=f'cooling_{cooling}',
            showlegend=True,
            hoverinfo='skip'
        ))
    
    fig.update_layout(
        title=plot_title,
        xaxis=dict(
            title='Training Compute (FLOP) [log scale]',
            type='log',
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=1
        ),
        yaxis=dict(
            title=f'{y_label} ({y_units}) [log scale]',
            type='log',
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=1
        ),
        width=1000,
        height=600,
        legend=dict(
            orientation='v',
            x=1.02,
            y=1,
            xanchor='left',
            yanchor='top',
            bgcolor='rgba(255,255,255,0.8)',
            itemsizing='constant',
            tracegroupgap=0,
            bordercolor='black',
            borderwidth=1,
            font=dict(size=10)
        ),
        margin=dict(r=300)  # Make room for legend
    )
    
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    
    return fig

In [ ]:
def create_multi_scatter_plot(data, y_column, y_label, y_units, y_title, plot_title):
    df = data.copy()
    
    unique_models = df['Model (Year)'].unique()
    unique_cooling = df['Cooling System'].unique()
    
    model_color_map = {model: f"hsl({i * 360 / len(unique_models)},70%,50%)" 
                      for i, model in enumerate(unique_models)}
    
    available_symbols = [
        'circle', 'x', 'square', 'diamond', 'cross', 'triangle-up', 'star'
    ]
    cooling_symbol_map = {cooling: available_symbols[i % len(available_symbols)] 
                         for i, cooling in enumerate(unique_cooling)}
    
    fig = go.Figure()

    energy_colors = {
        'Hydropower': 'blue',
        'Nuclear': 'red', 
        'CSP': 'green'
    }
    
    models_in_legend = set()
    
    for series_idx, (y_col, y_lab) in enumerate(zip(y_column, y_label)):
        for model in unique_models:
            model_data = df[df['Model (Year)'] == model]
            
            symbols = [cooling_symbol_map.get(cooling, 'circle') for cooling in model_data['Cooling System']]
            
            hover_text = []
            for _, row in model_data.iterrows():
                hover_text.append(f'<b>{model}</b><br>' +
                                f'Series: {y_lab}<br>' +
                                f'Cooling: {row["Cooling System"]}<br>' +
                                f'Training Compute: {row["Training compute (FLOP)"]:.2e} FLOP<br>' +
                                f'{y_lab}: {row[y_col]:.2e} {y_units}')
                
            if model not in models_in_legend:
                trace_name = model
                show_in_legend = True
                models_in_legend.add(model)
            else:
                trace_name = f"{model} - {y_lab}"
                show_in_legend = False
            
            fig.add_trace(go.Scatter(
                x=model_data['Training compute (FLOP)'],
                y=model_data[y_col],
                mode='markers',
                marker=dict(
                    color=model_color_map.get(model, '#000000'),
                    symbol=symbols,
                    size=8,
                    line=dict(width=0.5, color='white')
                ),
                name=trace_name,
                legendgroup=f'model_{model}',  # Group all traces for this model
                hovertemplate='%{hovertext}<extra></extra>',
                hovertext=hover_text,
                showlegend=show_in_legend
            ))


    for series_idx, (y_col, y_lab) in enumerate(zip(y_column, y_label)):
        valid_data = df.dropna(subset=[y_col, 'Training compute (FLOP)'])
        
        if len(valid_data) > 1:  # Need at least 2 points for regression
            # Convert to log space for linear regression
            log_x = np.log10(valid_data['Training compute (FLOP)'])
            log_y = np.log10(valid_data[y_col])
            
            # Perform linear regression in log space
            slope, intercept, r_value, p_value, std_err = stats.linregress(log_x, log_y)
            
            # Create regression line points
            x_range = np.logspace(np.log10(valid_data['Training compute (FLOP)'].min()), 
                                 np.log10(valid_data['Training compute (FLOP)'].max()), 100)
            y_regression = 10**(slope * np.log10(x_range) + intercept)
            
            # Add regression line
            fig.add_trace(go.Scatter(
                x=x_range,
                y=y_regression,
                mode='lines',
                line=dict(
                    color=energy_colors.get(y_lab, 'black'),
                    width=2,
                    dash='dash'
                ),
                name=f'{y_lab} Regression (R²={r_value**2:.3f})',
                hovertemplate=f'<b>{y_lab} Regression</b><br>' +
                             f'R² = {r_value**2:.3f}<br>' +
                             f'Slope = {slope:.3f}<br>' +
                             '<extra></extra>',
                showlegend=True
            ))

    
    for cooling in unique_cooling:
        fig.add_trace(go.Scatter(
            x=[None],
            y=[None],
            mode='markers',
            marker=dict(
                color='rgba(0,0,0,0.7)',
                symbol=cooling_symbol_map.get(cooling, 'circle'),
                size=8,
                line=dict(width=1, color='white')
            ),
            name=cooling,
            legendgroup=f'cooling_{cooling}',
            showlegend=True,
            hoverinfo='skip'
        ))
    
    fig.update_layout(
        title=plot_title,
        xaxis=dict(
            title='Training Compute (FLOP) [log scale]',
            type='log',
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=1
        ),
        yaxis=dict(
            title=f'{y_title} ({y_units}) [log scale]',
            type='log',
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=1
        ),
        width=1000,
        height=600,
        legend=dict(
            orientation='v',
            x=1.02,
            y=1,
            xanchor='left',
            yanchor='top',
            bgcolor='rgba(255,255,255,0.8)',
            itemsizing='constant',
            tracegroupgap=0,
            bordercolor='black',
            borderwidth=1,
            font=dict(size=10)
        ),
        margin=dict(r=300)  # Make room for legend
    )
    
    # Update axes to match matplotlib style
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    
    return fig

In [93]:
results = []
for _, row in top_notable_models.iterrows():
    model = row['Model']
    year = row['Year']
    hardware = row['Training hardware']
    flop = row['Training compute (FLOP)']
    fp32_flops, tdp = find_chip_info(hardware, year - 1, chip_df)

    results.append({
        'Model': model,
        'Year': year,
        'Training hardware': hardware,
        'Training compute (FLOP)' : flop,
        'FP32 GFLOPS': fp32_flops,
        'TDP (W)': tdp
    })

final_notable_models = pd.DataFrame(results)
final_notable_models.loc[0, ['FP32 GFLOPS', 'TDP (W)']] = [66910, 700]

final_notable_models['Energy (J)'] = (final_notable_models['TDP (W)'] / (final_notable_models['FP32 GFLOPS'] * 10**9)) * final_notable_models['Training compute (FLOP)']
final_notable_models['Energy (kWh)'] = final_notable_models['Energy (J)'] / (3.6e6)  # Convert Joules to kWh

combined_notable = combined_notable.merge(
    final_notable_models[['Model', 'Energy (kWh)']],
    on='Model',
    how='left'
)

In [94]:
combined_notable['Energy Per FLOP'] = combined_notable['Energy (kWh)'] / combined_notable['Training compute (FLOP)']

combined_notable['Cooling System Energy (kWh)'] = combined_notable['Energy (kWh)'] * (combined_notable['AvgPUE'] - 1)
combined_notable['Total Energy (kWh)'] = combined_notable['Energy (kWh)'] + combined_notable['Cooling System Energy (kWh)']
combined_notable['Total Energy Per FLOP'] = combined_notable['Total Energy (kWh)'] / combined_notable['Training compute (FLOP)']

combined_notable['Water Usage'] = combined_notable['AvgWUE'] * combined_notable['Energy (kWh)']
combined_notable['Water Usage Per FLOP'] = combined_notable['Water Usage'] / combined_notable['Training compute (FLOP)']

In [118]:
fig = create_scatter_plot(
    data=combined_notable,
    y_column='Energy Per FLOP',
    y_label='Energy Usage Per FLOP',
    y_units='kWh',
    plot_title='Compute Energy Usage (kWh) Per FLOP vs FLOP by Cooling System and Model'
)
fig.show()

In [119]:
fig = create_scatter_plot(
    data=combined_notable,
    y_column='Total Energy Per FLOP',
    y_label='Total Energy Usage Per FLOP',
    y_units='kWh',
    plot_title='Total Energy Usage (kWh) Per FLOP (liters/FLOP) vs FLOP by Cooling System and Model'
)
fig.show()

In [120]:
fig = create_scatter_plot(
    data=combined_notable,
    y_column='Water Usage Per FLOP',
    y_label='Water Usage Per FLOP',
    y_units='liters/FLOP',
    plot_title='Water Usage (liters) (Scope 1) Per FLOP vs FLOP by Cooling System and Model'
)
fig.show()

In [98]:
GAL_IN_LITERS = 3.78541

combined_notable['Water Usage (Scope 2) - Hydropower'] = 4.491 * combined_notable['Total Energy (kWh)'] * GAL_IN_LITERS
combined_notable['Total Water Usage - Hydropower'] = combined_notable['Water Usage'] + combined_notable['Water Usage (Scope 2) - Hydropower']
combined_notable['Water Usage (Scope 2) Per FLOP - Hydropower'] = combined_notable['Water Usage (Scope 2) - Hydropower'] / combined_notable['Training compute (FLOP)']
combined_notable['Total Water Usage Per FLOP - Hydropower'] = combined_notable['Total Water Usage - Hydropower'] / combined_notable['Training compute (FLOP)']

combined_notable['Water Usage (Scope 2) - CSP'] = 0.026 * combined_notable['Total Energy (kWh)'] * GAL_IN_LITERS
combined_notable['Total Water Usage - CSP'] = combined_notable['Water Usage'] + combined_notable['Water Usage (Scope 2) - CSP']
combined_notable['Water Usage (Scope 2) Per FLOP - CSP'] = combined_notable['Water Usage (Scope 2) - CSP'] / combined_notable['Training compute (FLOP)']
combined_notable['Total Water Usage Per FLOP - CSP'] = combined_notable['Total Water Usage - CSP'] / combined_notable['Training compute (FLOP)']

combined_notable['Water Usage (Scope 2) - Nuclear'] = 0.269 * combined_notable['Total Energy (kWh)'] * GAL_IN_LITERS
combined_notable['Total Water Usage - Nuclear'] = combined_notable['Water Usage'] + combined_notable['Water Usage (Scope 2) - Nuclear']
combined_notable['Water Usage (Scope 2) Per FLOP - Nuclear'] = combined_notable['Water Usage (Scope 2) - Nuclear'] / combined_notable['Training compute (FLOP)']
combined_notable['Total Water Usage Per FLOP - Nuclear'] = combined_notable['Total Water Usage - Nuclear'] / combined_notable['Training compute (FLOP)']

In [121]:
fig = create_multi_scatter_plot(
    data=combined_notable,
    y_column=['Total Water Usage Per FLOP - Hydropower', 'Total Water Usage Per FLOP - Nuclear', 'Total Water Usage Per FLOP - CSP'],
    y_label=['Hydropower', 'Nuclear', 'CSP'],
    y_units='liters',
    y_title='Total Water Usage Per FLOP (Hydro, Nuclear, CSP)',
    plot_title='Total Water Usage (liters) Per FLOP vs FLOP by Energy Source, Cooling System and Model'
)
fig.show()

In [100]:
carbon_df = carbon_df.rename(columns={'Carbon intensity of electricity - gCO2/kWh': 'Carbon Intensity (kg CO2 per kWh)'})
carbon_df['Carbon Intensity (kg CO2 per kWh)'] = carbon_df['Carbon Intensity (kg CO2 per kWh)'] / 1000
carbon_df_usa = carbon_df[carbon_df['Entity'] == 'United States'].copy()

intensity_2023 = carbon_df_usa.loc[carbon_df_usa['Year'] == 2023, 'Carbon Intensity (kg CO2 per kWh)'].values[0]
intensity_2024 = intensity_2023 * (1 - 0.025)
intensity_2025 = intensity_2024 * (1 - 0.025)

# Create DataFrame with new values
new_rows = pd.DataFrame({
    'Year': [2024, 2025],
    'Carbon Intensity (kg CO2 per kWh)': [intensity_2024, intensity_2025]
})

carbon_df_usa = pd.concat([carbon_df_usa, new_rows], ignore_index=True)

combined_notable = pd.merge(combined_notable, carbon_df_usa[['Year', 'Carbon Intensity (kg CO2 per kWh)']], on='Year', how='left')

combined_notable['Carbon Impact (kg CO2) - US Grid'] = combined_notable['Energy (kWh)'] * combined_notable['Carbon Intensity (kg CO2 per kWh)'] # US grid values
combined_notable['Cooling System Carbon Impact (kg CO2)'] = combined_notable['Cooling System Energy (kWh)'] * combined_notable['Carbon Intensity (kg CO2 per kWh)']
combined_notable['Total Carbon Impact (kg CO2) - US Grid'] = combined_notable['Total Energy (kWh)'] * combined_notable['Carbon Intensity (kg CO2 per kWh)']

combined_notable['Carbon Impact (kg CO2) Per FLOP - US Grid'] = combined_notable['Carbon Impact (kg CO2) - US Grid'] / combined_notable['Training compute (FLOP)']
combined_notable['Total Carbon Impact (kg CO2) Per FLOP - US Grid'] = combined_notable['Total Carbon Impact (kg CO2) - US Grid'] / combined_notable['Training compute (FLOP)']

In [101]:
combined_notable['Carbon Impact (kg CO2) - Hydropower'] = combined_notable['Energy (kWh)'] * 0.024
combined_notable['Carbon Impact (kg CO2) Per FLOP - Hydropower'] = combined_notable['Carbon Impact (kg CO2) - Hydropower'] / combined_notable['Training compute (FLOP)']
combined_notable['Total Carbon Impact (kg CO2) - Hydropower'] = combined_notable['Total Energy (kWh)'] * 0.024
combined_notable['Total Carbon Impact (kg CO2) Per FLOP - Hydropower'] = combined_notable['Total Carbon Impact (kg CO2) - Hydropower'] / combined_notable['Training compute (FLOP)']

combined_notable['Carbon Impact (kg CO2) - CSP'] = combined_notable['Energy (kWh)'] * 0.027
combined_notable['Carbon Impact (kg CO2) Per FLOP - CSP'] = combined_notable['Carbon Impact (kg CO2) - CSP'] / combined_notable['Training compute (FLOP)']
combined_notable['Total Carbon Impact (kg CO2) - CSP'] = combined_notable['Total Energy (kWh)'] * 0.027
combined_notable['Total Carbon Impact (kg CO2) Per FLOP - CSP'] = combined_notable['Total Carbon Impact (kg CO2) - CSP'] / combined_notable['Training compute (FLOP)']

combined_notable['Carbon Impact (kg CO2) - Nuclear'] = combined_notable['Energy (kWh)'] * 0.012
combined_notable['Carbon Impact (kg CO2) Per FLOP - Nuclear'] = combined_notable['Carbon Impact (kg CO2) - Nuclear'] / combined_notable['Training compute (FLOP)']
combined_notable['Total Carbon Impact (kg CO2) - Nuclear'] = combined_notable['Total Energy (kWh)'] * 0.012
combined_notable['Total Carbon Impact (kg CO2) Per FLOP - Nuclear'] = combined_notable['Total Carbon Impact (kg CO2) - Nuclear'] / combined_notable['Training compute (FLOP)']


In [116]:
fig = create_multi_scatter_plot(
    data=combined_notable,
    y_column=['Carbon Impact (kg CO2) Per FLOP - US Grid', 'Carbon Impact (kg CO2) Per FLOP - Hydropower', 'Carbon Impact (kg CO2) Per FLOP - CSP', 'Carbon Impact (kg CO2) Per FLOP - Nuclear'],
    y_label=['US Grid', 'Hydropower', 'Nuclear', 'CSP'],
    y_units='kg CO2',
    y_title='Carbon Impact Per FLOP (Hydro, Nuclear, CSP)',
    plot_title='Carbon Impact (kg CO2) Per FLOP vs FLOP by Energy Source, Cooling System and Model'
)
fig.show()

In [115]:
fig = create_multi_scatter_plot(
    data=combined_notable,
    y_column=['Total Carbon Impact (kg CO2) Per FLOP - US Grid', 'Total Carbon Impact (kg CO2) Per FLOP - Hydropower', 'Total Carbon Impact (kg CO2) Per FLOP - CSP', 'Total Carbon Impact (kg CO2) Per FLOP - Nuclear'],
    y_label=['US Grid', 'Hydropower', 'Nuclear', 'CSP'],
    y_units='kg CO2',
    y_title='Total Carbon Impact Per FLOP (Hydro, Nuclear, CSP)',
    plot_title='Total Carbon Impact (kg CO2) Per FLOP vs FLOP by Energy Source, Cooling System and Model'
)
fig.show()